In [14]:
import numpy as np
import random
import math
import networkx as nx

In [ ]:
class Agent:
    STATES = ('S', 'E', 'I', 'R')

    def __init__(self, state='S', days_left=1):
        self.state = state
        self.days_left = days_left

    def set_state(self, state, days_left):
        self.state = state
        self.days_left = days_left

    def advance_day(self) -> bool:
        if self.days_left > 0:
            self.days_left -= 1
        return self.days_left == 0

    def is_infectious(self) -> bool:
        return self.state == 'I'

In [ ]:
def got_infected():
    probability = ((0.12) / (1 - 0.12)) * math.exp(-0.0050367)
    return random.random() <= probability

In [12]:
def days_left_infected():
    return math.ceil(random.lognormvariate(2.25, 0.105))

In [13]:
def days_left_exposed():
    return math.ceil(random.lognormvariate(1.0, 1.0))


In [17]:
def is_node_infected(G, node):
    neighbors = G.neighbors(node)
    for neighbor in neighbors:
        if G.nodes[neighbor]['agent'].is_infectious():
            if(got_infected()):
                return True
            
    return False

In [ ]:
def get_starting_sickos(G, num: int):
    nodes = list(range(1, num + 1))
    random.shuffle(nodes)

    half = num * 0.05
    exposed = nodes[:half]
    infected = nodes[half : half * 2]
    sussy = nodes[half * 2 :]


    for node in exposed:
        G.nodes[node]['agent'].set_state('E', days_left_exposed())

    for node in infected:
        G.nodes[node]['agent'].set_state('I', days_left_infected())
    
    for node in sussy:
        G.nodes[node]['agent'].set_state('S', 0)

    return exposed, infected, sussy

In [ ]:
def stepI(G, infected):
    changed = []

    for node in infected:
        agent = G.nodes[node]['agent']
        if agent.advance_day():
            agent.set_state('R', 0)
            changed.append(node)

    return changed  

In [ ]:
def stepE(G, exposed):
    changed = []

    for node in exposed:
        agent = G.nodes[node]['agent']
        if agent.advance_day():
            agent.set_state('I', days_left_infected())
            changed.append(node)
    return changed

In [ ]:
def stepS(G, susceptible):
    changed = []
    for node in susceptible:
        agent = G.nodes[node]['agent']
        if is_node_infected(G, node):
            agent.set_state('E', days_left_exposed())
            changed.append(node)

    return changed